In [1]:
import polyfempy as pf
import numpy as np
import meshplot as mp
import igl

In [2]:
mesh_path = "box.mesh"
solver = pf.Solver()
solver.load_mesh_from_path(mesh_path)

[2020-02-20 11:25:37.152] [polyfem] [info] Loading mesh...
[2020-02-20 11:25:37.153] [geogram] [info] Loading file box.mesh...
[2020-02-20 11:25:37.158] [geogram] [info] (FP64) nb_v:125 nb_e:0 nb_f:192 nb_b:0 tri:1 dim:3
[2020-02-20 11:25:37.158] [geogram] [info]  nb_tets:384
[2020-02-20 11:25:37.159] [geogram] [info] Attributes on vertices: point[3]
[2020-02-20 11:25:37.166] [polyfem] [info] mesh bb min [0, 0, 0], max [0.2, 0.2, 0.2]
[2020-02-20 11:25:37.166] [polyfem] [info]  took 0.0140584s


In [3]:
p, t, s = solver.get_boundary_sidesets()

In [6]:
tmp = np.zeros_like(s)
tmp[s==1] = 1
tmp[s==3] = 2

mp.plot(p, t, tmp, return_plot=True)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.1000000…

In [25]:
order = 2

In [26]:
settings = pf.Settings(
    discr_order=order,
    pde=pf.PDEs.LinearElasticity,
)

settings = pf.Settings(
    discr_order=order,
    pde=pf.PDEs.NonLinearElasticity,
    
    nl_solver_rhs_steps=5
)

In [27]:
settings.set_material_params("E", 66000)
settings.set_material_params("nu", 0.49)

In [28]:
problem = pf.Problem()

In [29]:
problem.set_dirichlet_value(1, [0, 0, 0], [True, True, True])
problem.set_dirichlet_value(3, [-0.05, 0, 0], [True, True, True])

In [30]:
settings.problem = problem

#solving!
solver.settings(settings)

solver.solve()

[2020-02-20 15:00:00.612] [polyfem] [info] simplex_count: 	384
[2020-02-20 15:00:00.612] [polyfem] [info] regular_count: 	0
[2020-02-20 15:00:00.612] [polyfem] [info] regular_boundary_count: 	0
[2020-02-20 15:00:00.612] [polyfem] [info] simple_singular_count: 	0
[2020-02-20 15:00:00.612] [polyfem] [info] multi_singular_count: 	0
[2020-02-20 15:00:00.612] [polyfem] [info] boundary_count: 	0
[2020-02-20 15:00:00.612] [polyfem] [info] multi_singular_boundary_count: 	0
[2020-02-20 15:00:00.612] [polyfem] [info] non_regular_count: 	0
[2020-02-20 15:00:00.612] [polyfem] [info] non_regular_boundary_count: 	0
[2020-02-20 15:00:00.612] [polyfem] [info] undefined_count: 	0
[2020-02-20 15:00:00.612] [polyfem] [info] total count:	 384
[2020-02-20 15:00:00.613] [polyfem] [info] Building not isoparametric basis...
[2020-02-20 15:00:00.624] [polyfem] [info] Computing polygonal basis...
[2020-02-20 15:00:00.624] [polyfem] [info]  took 2.97046e-05s
[2020-02-20 15:00:00.626] [polyfem] [info] hmin: 0.05


In [31]:
pts, tets, disp = solver.get_sampled_solution(boundary_only=True)

vertices = pts + disp
mises, _ = solver.get_sampled_mises_avg(boundary_only=True)

In [32]:
mp.plot(vertices, tets, mises, shading={"flat":True}, return_plot=True)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0750000…

In [33]:
len = np.linalg.norm(disp, axis=1)

In [34]:
mp.plot(vertices, tets, len, shading={"flat":True}, return_plot=True)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0750000…

In [15]:
solver.export_vtu("box_results.vtu")